In [14]:
import csv
import functools
import gzip
import pandas as pd
import json
import tempfile
import gzip
import json
import tqdm
import mlflow
import torch.nn.functional as F

from gensim import corpora
from gensim.parsing import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from tqdm.notebook import tqdm, trange
from gensim.models import KeyedVectors
from torch.utils.data import Dataset, DataLoader, IterableDataset
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import bz2

from sklearn.metrics import classification_report


In [2]:
# #CORRER UNA VEZ
# !mkdir ./data
# !curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/glove.6B.50d.txt.gz -o ./data/glove.6B.50d.txt.gz
# !curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/meli-challenge-2019.tar.bz2 -o ./data/meli-challenge-2019.tar.bz2
# !tar jxvf ./data/meli-challenge-2019.tar.bz2 -C ./data/
# !pip install mlflow

In [17]:
!tar jxvf ./data/SBW-vectors-300-min5.txt.bz2 -C ./data/

tar: This does not look like a tar archive
tar: Skipping to next header
tar: Archive contains obsolescent base-64 headers
tar: Archive contains ‘-0.067809 -0’ where numeric off_t value expected
tar: Archive contains ‘104393 0’ where numeric mode_t value expected
tar: Archive contains ‘0.030046 -0.’ where numeric time_t value expected
tar: Archive contains ‘.119231 ’ where numeric uid_t value expected
tar: Archive contains ‘-0.00081’ where numeric gid_t value expected
7 0.032414 -0.066121 0.003187 -0.011452 -0.019466 0.043486 0.023361 0.034605 -0.054113 -0.031063 -0.
tar: 7 0.032414 -0.066121 0.003187 -0.011452 -0.019466 0.043486 0.023361 0.034605 -0.054113 -0.031063 -0.: Unknown file type '.', extracted as normal file
tar: 7 0.032414 -0.066121 0.003187 -0.011452 -0.019466 0.043486 0.023361 0.034605 -0.054113 -0.031063 -0.: implausibly old time stamp 1969-12-31 20:59:59
tar: Skipping to next header
^C


In [2]:
train_df = pd.concat([x for x in pd.read_json('./data/meli-challenge-2019/spanish.train.jsonl.gz', lines=True, chunksize=100000)], ignore_index=True)
train_df.drop(['language','label_quality','split', 'tokenized_title', 'data', 'target', 'n_labels', 'size'], inplace=True, axis=1)
train_df.head()

,title,category
0,Casita Muñecas Barbies Pintadas,DOLLHOUSES
1,Neceser Cromado Holográfico,TOILETRY_BAGS
2,Funda Asiento A Medida D20 Chevrolet,CAR_SEAT_COVERS
3,Embrague Ford Focus One 1.8 8v Td (90cv) Desde...,AUTOMOTIVE_CLUTCH_KITS
4,Bateria Panasonic Dmwbcf10 Lumix Dmc-fx60n Dmc...,CAMERA_BATTERIES


In [3]:
train_df_short = train_df[:200000]

In [4]:
train_df_short["longitud"] = train_df_short["title"].str.len()

/tmp/ipykernel_1309770/3038776986.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_short["longitud"] = train_df_short["title"].str.len()


In [5]:
train_df_short.sort_values("longitud", ascending = False)

,title,category,longitud
114106,Cooler Para Hp Pavilion Dm4 Dm4t Dm4-3000 Dm4-...,DESKTOP_COMPUTER_COOLERS_AND_FANS,120
170880,Cooler Para Hp Pavilion 15-p001nr 15-p010dx 15...,DESKTOP_COMPUTER_COOLERS_AND_FANS,120
124795,Bateria Para Gateway M-150 M-1615 M-1622 M-162...,LAPTOP_BATTERIES,120
128696,Kit Distribución Correa Y Tensor Original Gm +...,DISTRIBUTION_KITS,120
136254,Cooler Para Hp G7-1312nr G7-1321nr G7-1329wm G...,DESKTOP_COMPUTER_COOLERS_AND_FANS,120
...,...,...,...
4512,646,BABY_PLAYARDS,3
51167,966,ALL_IN_ONE,3
43217,Sdg,PEDAL_EFFECTS,3
199179,Sd1,MUSICAL_KEYBOARDS,3


In [6]:
test_df = pd.concat([x for x in pd.read_json('./data/meli-challenge-2019/spanish.test.jsonl.gz', lines=True, chunksize=100000)], ignore_index=True)
test_df.drop(['language','label_quality','split', 'tokenized_title', 'data', 'target', 'n_labels', 'size'], inplace=True, axis=1)
test_df

,title,category
0,Mochilas Maternales Bolsos Bebe Simil Cuero Ma...,DIAPER_BAGS
1,Bolso Maternal/bebe Incluye Cambiador + Correa...,DIAPER_BAGS
2,Mochila Maternal Land + Gancho Envio Gratis-cc,DIAPER_BAGS
3,Bolso Maternal Moderno Con Cambiador Y Correa ...,DIAPER_BAGS
4,Bolso Maternal Moderno Con Cambiador Y Correa ...,DIAPER_BAGS
...,...,...
63675,Gimnasio Gym Manta Bebe Tiny Love Musica Luz M...,BABY_GYMS
63676,Gimnasio Manta Con Actividades Para Bebe 846 Ath,BABY_GYMS
63677,Gimnasio Bebe Manta Didactica Tiny Love Kick A...,BABY_GYMS
63678,Gimnasio Manta Alfombra Didactica Fitchbaby Ju...,BABY_GYMS


In [6]:
# test_df_short = test_df[:100000]

In [20]:
class RawDataProcessor:
    def __init__(self, 
                 dataset, 
                 ignore_header=True, 
                 filters=None, 
                 vocab_size=50000):
        if filters:
            self.filters = filters
        else:
            self.filters = [ #We set some filters
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        # Create dictionary based on all the reviews (with corresponding preprocessing and filters)
        # The dictionary has idx as a key and word as a value
        # For example one element could be (0, 'accustomed') or (1, 'agenda')
        self.dictionary = corpora.Dictionary(
            dataset["title"].map(self._preprocess_string).tolist()
        )
        
        # Filter the dictionary and compactify it (make the indices continous)
        self.dictionary.filter_extremes(no_below=2, no_above=1, keep_n=vocab_size)
        self.dictionary.compactify()
        # Add a couple of special tokens
        self.dictionary.patch_with_special_tokens({
            "[PAD]": 0, #The padding token
            "[UNK]": 1  # The unknown token
        })
        
        self.idx_to_target = sorted(dataset["category"].unique())
        self.target_to_idx = {t: i for i, t in enumerate(self.idx_to_target)}
        

    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _sentence_to_indices(self, sentence):
        return self.dictionary.doc2idx(sentence, unknown_word_index=1)
    
    def encode_data(self, data):
        return self._sentence_to_indices(self._preprocess_string(data))
    
    def encode_target(self, target):
        return self.target_to_idx[target]
    
    def __call__(self, item):
        #Encodeamos tanto los datos como los targets, diferenciandos los casos cuando son strings y cuando no
        if isinstance(item["data"], str): 
            data = self.encode_data(item["data"])
        else:
            data = [self.encode_data(d) for d in item["data"]]
        
        if isinstance(item["target"], str):
            target = self.encode_target(item["target"])
        else:
            target = [self.encode_target(t) for t in item["target"]]
        
        return {
            "data": data,
            "target": target
        }

In [21]:
class MyDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
    
    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        if torch.is_tensor(item):
            item = item.to_list()
        
        item = {
            "data": self.dataset.loc[item, "title"],
            "target": self.dataset.loc[item, "category"]
        }
        
        if self.transform:
            item = self.transform(item)
        
        return item

In [22]:
preprocess_train = RawDataProcessor(train_df_short)
train_dataset = MyDataset(train_df_short, transform=preprocess_train)

preprocess_test = RawDataProcessor(test_df)
test_dataset = MyDataset(test_df, transform=preprocess_test)

print(f"Datasets loaded with {len(train_dataset)} training elements and {len(test_dataset)} test elements")
print(f"Sample train element:\n{train_dataset[0]}")

Datasets loaded with 200000 training elements and 63680 test elements
Sample train element:
{'data': [36328, 2, 36327, 3], 'target': 188}


In [23]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
            "data": torch.LongTensor(data),
            "target": torch.FloatTensor(target)
        }

In [24]:
pad_sequences = PadSequences()

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,collate_fn=pad_sequences, drop_last=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,collate_fn=pad_sequences, drop_last=False, num_workers=0)

In [25]:
class MyClassifier(nn.Module):
    def __init__(self, 
                 pretrained_embeddings_path, 
                 dictionary,
                 vector_size,
                 freeze_embedings):
        super().__init__()
        embeddings_matrix = torch.randn(len(dictionary), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        with bz2.open(pretrained_embeddings_path, "rt") as fh:
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        self.hidden1 = nn.Linear(vector_size, 120)
        self.hidden2 = nn.Linear(120, 60)
        self.output = nn.Linear(60, 632)
        self.vector_size = vector_size
    
    def forward(self, x):
        x = self.embeddings(x)
        x = torch.mean(x, dim=1)
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        
        x = torch.sigmoid(self.output(x))
        
        return x

In [26]:
mlflow.set_experiment("Experiment_01")
EPOCHS = 2
with mlflow.start_run():
    mlflow.log_param("model_name", "mlp")
    mlflow.log_param("freeze_embedding", True)
    mlflow.log_params({
        "embedding_size": 120,
        "hidden1_size": 120,
        "hidden2_size": 60
    })
    model = MyClassifier("./data/SBW-vectors-300-min5.txt.bz2", preprocess_train.dictionary, 300, True)
    loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    for epoch in range(EPOCHS):
        model.train()
        running_loss = []
        for idx, batch in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            output = model(batch["data"])
            loss_value = loss(output, batch["target"].squeeze().long())
            loss_value.backward()
            optimizer.step()
            running_loss.append(loss_value.item())        
        mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)
        
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        for idx, batch in enumerate(tqdm(train_loader)):

            output = model(batch["data"])
            running_loss.append(
                loss(output, batch["target"].squeeze().long()).item()
            )
            targets.extend(batch["target"].numpy())
            predictions.extend(output.squeeze().detach().numpy())
        mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epoch)
#         mlflow.log_metric("test_avp", average_precision_score(targets, predictions), epoch)
        
#     with tempfile.TemporaryDirectory() as tmpdirname:
#         targets = []
#         predictions = []
#         for idx, batch in enumerate(tqdm(train_loader)):
#         #for batch in tqdm(test_loader):
#             output = model(batch["data"])
#             targets.extend(batch["target"].numpy())
#             predictions.extend(output.squeeze().detach().numpy())
# #             df_predict = pd.DataFrame({"prediction": predictions, "target": targets})
#         pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
#             f"{tmpdirname}/predictions.csv.gz", index=False
#         )
#         mlflow.log_artifact(f"{tmpdirname}/predictions.csv.gz")

  0%|          | 0/6250 [00:00<?, ?it/s]

  0%|          | 0/6250 [00:00<?, ?it/s]

  0%|          | 0/6250 [00:00<?, ?it/s]

  0%|          | 0/6250 [00:00<?, ?it/s]

In [27]:
print(model)

MyClassifier(
  (embeddings): Embedding(36329, 300, padding_idx=0)
  (hidden1): Linear(in_features=300, out_features=120, bias=True)
  (hidden2): Linear(in_features=120, out_features=60, bias=True)
  (output): Linear(in_features=60, out_features=632, bias=True)
)


In [28]:
mlflow.tracking.MlflowClient().list_experiments()

[<Experiment: artifact_location='file:///users/lguedez/AprendizajeProfundo/Entregable/mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='file:///users/lguedez/AprendizajeProfundo/Entregable/mlruns/1', experiment_id='1', lifecycle_stage='active', name='Experiment_01', tags={}>]

In [29]:
mlflow.search_runs()
# mlflow.search_runs(experiment_ids='1', filter_string= 'metrics.score_iris > 0.1').head()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.test_loss,metrics.train_loss,params.freeze_embedding,params.model_name,params.hidden2_size,params.embedding_size,params.hidden1_size,params.hidden3_size,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.source.name
0,73b732ea410e4ad7adc3ebf3676d0860,1,FINISHED,file:///users/lguedez/AprendizajeProfundo/Entr...,2022-10-11 23:35:58.677000+00:00,2022-10-11 23:42:11.053000+00:00,5.885289,5.930865,True,mlp,60,120,120,None,lguedez,LOCAL,/users/lguedez/miniconda3/envs/deeplearning/li...
1,453ee428925f49f48ecf137783ee1f36,1,FAILED,file:///users/lguedez/AprendizajeProfundo/Entr...,2022-10-11 23:34:46.930000+00:00,2022-10-11 23:35:24.190000+00:00,NaN,NaN,True,mlp,60,120,120,None,lguedez,LOCAL,/users/lguedez/miniconda3/envs/deeplearning/li...
2,8e987d88086b46109abf815a9cdae097,1,FAILED,file:///users/lguedez/AprendizajeProfundo/Entr...,2022-10-11 23:31:04.506000+00:00,2022-10-11 23:31:04.605000+00:00,NaN,NaN,True,mlp,60,120,120,None,lguedez,LOCAL,/users/lguedez/miniconda3/envs/deeplearning/li...
3,fbfbe476a3134dbebc87d3ad61e57ed9,1,FAILED,file:///users/lguedez/AprendizajeProfundo/Entr...,2022-10-11 23:28:29.005000+00:00,2022-10-11 23:28:29.117000+00:00,NaN,NaN,True,mlp,60,120,120,None,lguedez,LOCAL,/users/lguedez/miniconda3/envs/deeplearning/li...
4,14ba2dca0fdf4b4d9a71eb0bef42dd6f,1,FAILED,file:///users/lguedez/AprendizajeProfundo/Entr...,2022-10-11 22:24:07.386000+00:00,2022-10-11 22:44:38.167000+00:00,NaN,NaN,True,mlp,128,100,128,128,lguedez,LOCAL,/users/lguedez/miniconda3/envs/deeplearning/li...
5,b8f03b23576b493886cb935c1e03bfd1,1,FINISHED,file:///users/lguedez/AprendizajeProfundo/Entr...,2022-10-11 22:11:58.735000+00:00,2022-10-11 22:14:28.382000+00:00,5.953580,5.992475,True,mlp,128,100,128,128,lguedez,LOCAL,/users/lguedez/miniconda3/envs/deeplearning/li...
6,28d42964f7624d518097ee494cc5d540,1,FAILED,file:///users/lguedez/AprendizajeProfundo/Entr...,2022-10-11 21:50:30.786000+00:00,2022-10-11 22:02:14.741000+00:00,5.823593,5.867229,True,mlp,128,50,128,None,lguedez,LOCAL,/users/lguedez/miniconda3/envs/deeplearning/li...
7,97bccba22edf4e4bb7ab029fddd6a4bc,1,FAILED,file:///users/lguedez/AprendizajeProfundo/Entr...,2022-10-11 21:44:18.709000+00:00,2022-10-11 21:50:17.042000+00:00,5.817949,5.858608,True,mlp,128,50,128,None,lguedez,LOCAL,/users/lguedez/miniconda3/envs/deeplearning/li...
8,2446bc8958624d338410f8c59e2941b9,1,FINISHED,file:///users/lguedez/AprendizajeProfundo/Entr...,2022-10-08 01:13:25.881000+00:00,2022-10-08 01:50:29.897000+00:00,5.818737,5.861299,True,mlp,128,50,128,None,lguedez,LOCAL,/users/lguedez/miniconda3/envs/deeplearning/li...
9,6f5546b5886e49c2b76f6739d92df7a9,1,FINISHED,file:///users/lguedez/AprendizajeProfundo/Entr...,2022-10-08 00:55:56.032000+00:00,2022-10-08 00:57:55.639000+00:00,NaN,5.858512,True,mlp,128,50,128,None,lguedez,LOCAL,/users/lguedez/miniconda3/envs/deeplearning/li...
